In [19]:
using Plots
using Printf
using CPUTime
using LinearAlgebra


function matvecmulti_gen(n::Int64)

    """Generalized matrix-vector multiplication with a tridiagonal matrix"""

    h = 1.0/(n + 1)          # step size
    hh = 100*h*h
    a = zeros(Float64, n)
    b = zeros(Float64, n)
    c = zeros(Float64, n)
    d = zeros(Float64, n)

    v = zeros(Float64, n)

    # initialize

    for i = 1:n
        a[i] = -1.0
        b[i] = 2.0
        c[i] = -1.0
        d[i] = hh*exp(-10.0*(i*h))  # source term
    end

    a[1] = 0.0
    c[n] = 0.0


    # forward substitution, t: CPU time used
    t = @elapsed @CPUelapsed for i = 2:n
        w = a[i]/b[i-1]

        b[i] -= w*c[i-1]
        d[i] -= w*d[i-1]
    end

    # backward substitution
    t += @elapsed @CPUelapsed v[n] = d[n]/b[n]

    t += @elapsed @CPUelapsed for i = n-1:-1:1
        v[i] = (d[i] - c[i]*v[i+1])/b[i]
    end

    return v, t

end


function exact(x::Float64)
    """ Analytical solution """
    return 1 - (1 - exp(-10))*x - exp(-10*x)
end


function plotting(n::Int64)
    """A function for plotting the numerical vs. analyical solution
    with varying number of grid points"""

    x = range(0, stop=1, length=n+2) # x-values
    y = matvecmulti_gen(n)[1]        # numerical values from the general algorithm
    y = vcat(0, y, 0)                # add the boundary values (concatenate)

    # plotting the numerical and analytical solution in the same figure
    plot(x, y, title="n = $(n)", label="Numerical", xlabel="x", ylabel="v(x)")
    plot!(x, map(exact, x), label="Analytical")

    # save figure
    savefig("1bplot_n=$(n).png")

end


function matvecmulti_spec(n::Int64)
    """
        Matrix-vector multiplication with a tridiagonal Toeplitz matrix
        n: number of grid points
    """


    h = 1.0/(n + 1)          # step size
    hh = 100*h*h
    d = zeros(Float64, n)    # right side of equation
    b = zeros(Float64, n)    # array for storing 1/b[i]
    v = zeros(Float64, n)    # array for storing result

    # initialize source term and b
    d = [hh*exp(-10.0*(i*h)) for i = 1:n]
    b = [1.0/((i+1)/i) for i = 1:n]

    # forward substitution
    t = @elapsed @CPUelapsed  for i = 2:n
        d[i] += d[i-1]*b[i-1]
    end

    # backward substitution
    t += @elapsed @CPUelapsed  v[n] = d[n]*b[n]

    t += @elapsed @CPUelapsed  for i = n-1:-1:1
        v[i] = (d[i+1] + v[i+1])*b[i]
    end

    return v, t

end

function CPU_time(n::Int64)


    t_gen = matvecmulti_gen(n)[2]
    t_spec = matvecmulti_spec(n)[2]

    @printf("General algorithm time with %d grid points: %lf ms\n", n, t_gen*1000)
    @printf("Special algorithm time with %d grid points: %lf ms\n", n, t_spec*1000)
    @printf("Relative difference: %lf\n", (t_gen/t_spec))
    @printf("\n")

    return t_gen, t_spec

end


function rel_error(n::Int64)

    h = 1.0/(n+1)
    x = range(h, stop=h*n, length=n)

    # calculate the numerical and analytical solutions
    v = matvecmulti_gen(n)[1]
    u = map(exact, x)

    # calculate the absolute difference between each element
    diff = (v .- u) ./ u


    abs_value = broadcast(abs, diff)
    epsilon = maximum(broadcast(log10, abs_value))

    return epsilon
end

function plot_error(n=1:8)

    err = zeros(length(n)) # array for storing error values

    #calculate, store, and print error values
    for (index, value) in enumerate(n)
        err[index] = rel_error(10^value)
        @printf("n = 10^%d\n", value)
        @printf("log10 relative error: epsilon = %lf\n", err[index])
        @printf("\n")
    end

    # plotting

    x = n
    plot(x, err, xlabel="log10 n", ylabel="log10 max epsilon",
        title="Maximum error as a function of # grid points",
        legend=false)
    scatter!(x, err, legend=false)
    #savefig("max_error.png")

end


function compare_LU(n::Int64)
    """
    A function for timing the built-in lu-factorization function in Julia' linear algebra module
    """

    # build the tridiagonal matrix
    A = zeros(n, n)
    for i = 1:n
        for j = 1:n
            if (j == i-1) || (j == i+1)
                A[i, j] = -1
            elseif (j == i)
                A[i,j] = 2
            end
        end
    end


    t = @elapsed @CPUelapsed lu(A)

    @printf("Julia LU factorization time with %d grid points:  %lf ms\n", n, t*1000)
    @printf("\n")
end


function main(n::Int64, ploterror=false)

    #plotting(n)
    CPU_time(n)
    compare_LU(n)

    if ploterror
        plot_error()
    end

end

main (generic function with 2 methods)

In [21]:
n = [10, 100, 1000]

for i in n
    main(i)
end

General algorithm time with 10 grid points: 0.037300 ms
Special algorithm time with 10 grid points: 0.029000 ms
Relative difference: 1.286207

Julia LU factorization time with 10 grid points:  0.040700 ms

General algorithm time with 100 grid points: 0.237200 ms
Special algorithm time with 100 grid points: 0.066199 ms
Relative difference: 3.583136

Julia LU factorization time with 100 grid points:  1.851800 ms

General algorithm time with 1000 grid points: 0.051299 ms
Special algorithm time with 1000 grid points: 0.036301 ms
Relative difference: 1.413157

Julia LU factorization time with 1000 grid points:  42.012200 ms



In [25]:
compare_LU(10^4)

Julia LU factorization time with 10000

InterruptException: InterruptException: